# Evaluate on TotalSegmentator MRI Dataset

## 1. Download data

```bash
# download data (2.4 GB)
mkdir TotalSegmentatorMRI; cd TotalSegmentatorMRI
wget https://zenodo.org/records/11367005/files/TotalsegmentatorMRI_dataset_v100.zip?download=1 -O TotalSegmentatorMRI.zip

unzip TotalSegmentatorMRI.zip
```

## 2. Merge annotations into single file
Execute this file in the current directory:

```bash
python merge_total_mri_annotations.py <path to totalsegmentatot-mri data>
```

# 3. Extract and analyze the test set
The testset consists of 30 MRI scans.
Six of these scans do not have a groundtruth (i.e. hand, breast) or include classes not supported by MRSegmentator (i.e. brain, shoulder, lower limb).

Of those we remove image s0129 (brain) from the test set, because it lacks orthonormal definition in the file header, a requirement for MRSegmentator.
The other five scans are kept in the evaluation pipe, but, by definiton, will not have any impact on the calculation of the DSC.

Of the 24 images that we use to calculate the DSC, 16 are axial, 5 sagittal and 3 coronal.




```bash 
grep test meta.csv  | awk -F ';' '{print $1}' > test_ids.txt
mkdir image_links label_links

cd image_links
for f in $(cat ../test_ids.txt)
do 
    ln -s ../$f/mri.nii.gz $f.nii.gz
done
unlink s0129.nii.gz

cd ../label_links
for f in $(cat ../test_ids.txt)
do 
    ln -s ../$f/annotation_merged.nii.gz $f.nii.gz
done
unlink s0129.nii.gz
cd ..
```


## 4. Run inference

```bash
mrsegmentator -i image_links --outdir preds
```

## 5. Evaluate

In [1]:
import pandas as pd
import eval_data
import mappings
from os.path import join

data_path = "/home/user/totalsegmentator_mri"

eval_data.main(
    label_dir=join(data_path, "label_links"),
    seg_dir=join(data_path, "preds"),
    dataset_name="csvs/TotalSegentator-MRI",
    class_mapping=mappings.total_to_mrseg,
)

data = pd.read_csv("csvs/TotalSegentator-MRI_dsc.csv").mean()

pd.options.display.float_format = "{:,.2f}".format
print("Mean:", round(data.mean(), 2))
data

Mean: 0.74


spleen                         0.89
right_kidney                   0.89
left_kidney                    0.77
gallbladder                    0.84
liver                          0.82
stomach                        0.85
pancreas                       0.44
right_adrenal_gland            0.59
left_adrenal_gland             0.65
left_lung                      0.92
right_lung                     0.82
heart                          0.81
aorta                          0.69
inferior_vena_cava             0.65
portal_vein_and_splenic_vein   0.62
left_iliac_artery              0.66
right_iliac_artery             0.63
left_iliac_vena                0.76
right_iliac_vena               0.74
esophagus                      0.62
small_bowel                    0.69
duodenum                       0.67
colon                          0.82
urinary_bladder                0.90
spine                          0.68
sacrum                         0.79
left_hip                       0.69
right_hip                   

## 6. What about scans from other body regions?
As written above we segmented five scans from other body regions, that didn't have any of MRSegmentor's classes. 
The segmented results can be seen below.
![image](../images/scans_of_other_regions.png)

Ideally, all segmentations should be empty. Indeed, this is the case for images s0027 and s0078. 
However, in the other three scans small halucinations can be detected: In s0150 parts of the Fibia are segmented as Femur. 
In s0145 tiny pieces of the shoulder are marked as spine. Lastly, in s0109 some voxels are marked as "heart" in an obscured view of the thorax. (Actually, the heart should be somewhere around this region, but it can not be seen in this scan. Seems to be an edge case?)

To put it short, don't use the model for regions that it wasn't trained for.
